In [51]:
import pandas as pd
import numpy as np
from typing import Dict, List

In [19]:
df = pd.read_csv("Api/Data/EURUSD_1m.csv", nrows=1000)

In [15]:
def interval_to_minutes(interval: str) -> int:
    """
    Convert an interval to the number of minutes\n

    Format: {amount}{time scale} e.g. 15m, 1h, 1D,...
    
    Args:
        interval (str): Inteval of time (15m, 1h, 1D)

    Returns:
        int: Number of minutes in interval (1h = 60m)
    """
    
    scales: Dict[str: int] = {
        "m": 1,
        "h": 60,
        "D": 1440,
        "W": 10080
    }
    
    
    multiplyer = int(interval[:-1])
    scale = interval[-1]
    
    if scale not in scales.keys():
        raise KeyError("Invalid time scale. Valid scales: m (minutes), h (hours), D (days), M (months)")
    
    else:
        return multiplyer * scales[scale]
    

In [68]:
def defractalise_ohlc(df: pd.DataFrame, interval: str):
    n_candles = interval_to_minutes(interval=interval)
    
    ohlc_data: List[Dict] = []
    for i in range(1, df.shape[0], n_candles):
        
        try:
            new_candle = df.iloc[i:i+n_candles]
        
        except IndexError:
            break
        
        ohlc_data.append({
            "Time": new_candle.iloc[-1]["Time"],
            "Open":  new_candle.iloc[0]["Open"],
            "High":  new_candle["High"].max(),
            "Low":  new_candle["Low"].min(),
            "Close": new_candle.iloc[-1]["Close"],
            "Volume": round(new_candle["Volume"].mean(), 2),
        })
        

        
    return pd.DataFrame(ohlc_data, columns=df.columns)